In order to obtain the monthly profit growth of merchants, we should compute the monthly income for each merchant. Therefore we must read 

In [2]:
from pyspark.sql import functions as F
from pyspark.sql.functions import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [3]:
from pyspark.sql import SparkSession

# Create a spark session
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .getOrCreate()
)

22/10/07 18:25:07 WARN Utils: Your hostname, LAPTOP-UQ7BDL4T resolves to a loopback address: 127.0.1.1; using 172.23.116.160 instead (on interface eth0)
22/10/07 18:25:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/07 18:25:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
period1 = pd.date_range('2021-2-28','2021-08-27', freq='D').strftime("%Y-%m-%d").tolist()
period2 = pd.date_range('2021-8-28','2022-02-27', freq='D').strftime("%Y-%m-%d").tolist()
period3 = pd.date_range('2022-2-28','2022-08-28', freq='D').strftime("%Y-%m-%d").tolist()
path_prefix1 = '../data/tables/transactions_20210228_20210827_snapshot/order_datetime='
path_prefix2 = '../data/tables/transactions_20210828_20220227_snapshot/order_datetime='
path_prefix3 = '../data/tables/transactions_20220228_20220828_snapshot/order_datetime='


In [5]:
# add a column of date
from pyspark.sql.functions import lit

In [6]:
dfSchema = spark.read.parquet('../data/tables/transactions_20210228_20210827_snapshot/order_datetime=2021-02-28')
dfSchema.schema

StructType([StructField('user_id', LongType(), True), StructField('merchant_abn', LongType(), True), StructField('dollar_value', DoubleType(), True), StructField('order_id', StringType(), True)])

In [7]:
# Create a empty dataframe to 

from pyspark.sql.types import StructType,StructField, StringType, LongType, DoubleType, IntegerType
emptyRDD = spark.sparkContext.emptyRDD()
schema = StructType([StructField('user_id', LongType(), True), StructField('merchant_abn', LongType(), True), StructField('dollar_value', DoubleType(), True), StructField('order_id', StringType(), True), StructField('date', IntegerType(), False)])
transaction_df_with_date = spark.createDataFrame(emptyRDD,schema)


For each transaction, add its date as a feature.

In [8]:

for date in period1:

    tep = spark.read.parquet(path_prefix1 + date)
    
    tep = tep.withColumn("date", lit(date))
    transaction_df_with_date = transaction_df_with_date.union(tep)
    


In [9]:
for date in period2:

    tep = spark.read.parquet(path_prefix2 + date)
    
    tep = tep.withColumn("date", lit(date))
    transaction_df_with_date = transaction_df_with_date.union(tep)

for date in period3:

    tep = spark.read.parquet(path_prefix3 + date)
    
    tep = tep.withColumn("date", lit(date))
    transaction_df_with_date = transaction_df_with_date.union(tep)

Obtain the month of the date for each transaction.

In [10]:
transaction_df_with_month = transaction_df_with_date.select(F.date_format('date','yyyy-MM').alias('month'),'merchant_abn','dollar_value')

Group by month to obtain the monthly income for each merchant.

In [11]:
monthly_merchant_df = transaction_df_with_month.groupby('month','merchant_abn').agg(sum('dollar_value').alias('monthly_value'))

In [12]:
monthly_merchant_df.write.mode('overwrite').parquet('../data/curated/monthly_merchant_detail.parquet')

22/10/07 18:27:39 WARN DAGScheduler: Broadcasting large task binary with size 3.4 MiB


22/10/07 18:28:28 WARN DAGScheduler: Broadcasting large task binary with size 1406.3 KiB


22/10/07 18:28:29 WARN MemoryManager: Total allocation exceeds 95.00% (947,335,975 bytes) of heap memory
Scaling row group sizes to 88.23% for 8 writers
